In [ ]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, singularity_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(singularity_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint)
from singularity_helper import (generate_singularity_topology, generate_biaxial_grid)
import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
edge_length = 15
cam_params = ((2.0787438565755934, 4.110669876049863, -1.9445352012461516),
 (-0.3012591112717844, -0.2789859717380453, -0.9118167444443614),
 (0.0, 0.0, 0.0))

### Triaxial Singularity Patches

In [ ]:
polygon_size = 6

In [ ]:
generate_singularity_topology(polygon_size)

In [ ]:
rod_length = 1
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('singularity_{}'.format(polygon_size), thickness, width, weaving_dir)

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 10000
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = None, useCenterline = True, model_path = 'singularity_patches/{}_gon.obj'.format(polygon_size), cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=True)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view.show()

In [ ]:
curved_linkage.numSegments()

In [ ]:
psrl = np.ones(curved_linkage.numSegments())

Triaxial Geometric Singularity - Positive Gaussian

In [ ]:
# psrl[:6] *= 1.4
# psrl[6:18] *= 1.35
# psrl[18:36] *= 1.3
# psrl[36:48] *= 1.2
# psrl[48:72] *= 1.15
# psrl[72:] *= 0.9

Triaxial Geometric Singularity - Negative Gaussian

In [ ]:
# psrl[:6] *= 1/1.4
# psrl[6:18] *= 1/1.35
# psrl[18:36] *= 1/1.3
# psrl[36:48] *= 1/1.2
# psrl[48:72] *= 1/1.15
# psrl[72:] *= 1/0.9
# psrl *= 1.1

In [ ]:
bottomColor =[79/255., 158/255., 246/255.]
topColor =[0.5, 0.5, 0.5]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [ ]:
curved_linkage.setPerSegmentRestLength(psrl)

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField=colors)

In [ ]:
driver=curved_linkage.centralJoint()
jdo = curved_linkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with so(): elastic_rods.compute_equilibrium(curved_linkage, fixedVars=fixedVars, callback = eqm_callback, options = OPTS)

In [ ]:
curved_linkage_view.update(scalarField=colors)

In [ ]:
save_path = os.getcwd()
write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME + '_geometric_irregularity_positive', io.RIBBON_NAME, False, 1, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS), use_surface = False)
os.chdir(save_path)

### Biaxial

In [ ]:
grid_size = 4

In [ ]:
def generate_biaxial_grid(grid_size):
    unit_width = 1
    vertices = []
    edges = []

    for i in range(grid_size):
        for j in range(grid_size):
            vertices.append([i, j, 0])
    
    for i in range(grid_size):
        vertices.append([-1, i, 0])
        vertices.append([grid_size, i, 0])
    for i in range(grid_size):
        vertices.append([i, -1, 0])
        vertices.append([i, grid_size, 0])

    for i in range(grid_size):
        for j in range(grid_size - 1):
            edges.append([i * grid_size + j, i * grid_size + j + 1])
            
    for i in range(grid_size):
        for j in range(grid_size - 1):
            edges.append([i + grid_size * j, i  + grid_size * (j + 1)])
    
    for i in range(grid_size):
        edges.append([grid_size ** 2 + i * 2, i])
        edges.append([grid_size ** 2 + i * 2 + 1, grid_size ** 2 - grid_size + i])
        
        edges.append([grid_size ** 2 + grid_size * 2 + i * 2, i * (grid_size)])
        edges.append([grid_size ** 2 + grid_size * 2 + i * 2 + 1, i * (grid_size) + grid_size - 1])
        

    print(edges)
    biaxial_path = 'biaxial'
    if not os.path.exists(biaxial_path):
        os.makedirs(biaxial_path)

    with open ('{}/{}_grid.obj'.format(biaxial_path, grid_size), 'w') as f:
        for point in vertices:
            f.write('v {} {} {}\n'.format(point[0], point[1], point[2]))
        for edge in edges:
            f.write('l {} {}\n'.format(edge[0]+1, edge[1]+1))


In [ ]:
generate_biaxial_grid(grid_size)

In [ ]:
rod_length = 1
width = rod_length / 20 * 5
# thickness = width / 1 * 0.35
thickness = width 
# thickness = 0.017499999999999998
print(width, thickness)
io = InputOrganizer('biaxial_{}'.format(grid_size), thickness, width, weaving_dir)

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 10000
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = None, useCenterline = True, model_path = 'biaxial/{}_grid.obj'.format(grid_size), cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=True, interleaving_type=InterleavingType.triaxialWeave)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view.show()

In [ ]:
curved_linkage.numSegments()

In [ ]:
psrl = np.ones(curved_linkage.numSegments())

In [ ]:
topColor =[79/255., 158/255., 246/255.]
bottomColor =[0.5, 0.5, 0.5]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [ ]:
curved_linkage.setPerSegmentRestLength(psrl)

In [ ]:
equilibrium_data = []
def eqm_callback(prob, i):
    equilibrium_data.append([curved_linkage.energy(), curved_linkage.energy(elastic_rods.EnergyType.Bend), curved_linkage.energy(elastic_rods.EnergyType.Twist), curved_linkage.energy(elastic_rods.EnergyType.Stretch)])
    curved_linkage_view.update(scalarField=colors)

In [ ]:
curved_linkage.energy(elastic_rods.EnergyType.Bend)

In [ ]:
driver=curved_linkage.centralJoint()
jdo = curved_linkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
elastic_rods.compute_equilibrium(curved_linkage, fixedVars=fixedVars, callback = eqm_callback, options = OPTS)

In [ ]:
equilibrium_data = np.array(equilibrium_data)

In [ ]:
fig, host = plt.subplots()
plt.plot(equilibrium_data[0:])
leg = host.legend(['Total', 'Bend', 'Twist', 'Stretch'], loc="center right", facecolor='white', framealpha=1, fancybox=True, prop={'size': 15})
fig.savefig('partial_energy.png', dpi=200)

In [ ]:
curved_linkage_view.update(scalarField=colors)

In [ ]:
save_path = os.getcwd()
write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME, io.RIBBON_NAME, False, 1, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS), use_surface = False)
os.chdir(save_path)